In [1]:
import pandas as pd
import requests
import numpy as np
import os
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

In [2]:
os.listdir("../../data")

['.DS_Store',
 'Acquisition.ipynb',
 'bicimad_stations.json',
 '209426-0-templos-catolicas.csv',
 '209434-0-templos-otros.csv',
 'bicipark_stations.json',
 '.ipynb_checkpoints']

In [3]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [4]:
def distance_meters(start, final):
    # return the distance in metres between to latitude/longitude pair point in degrees 
    return start.distance(final)

In [5]:
def templos_catolicos():
    templos_catolicos = requests.get('https://datos.madrid.es/egob/catalogo/209426-0-templos-catolicas.json')
    templos_catolicos = templos_catolicos.json()
    df1 = pd.json_normalize(templos_catolicos['@graph'])
    df1 = df1.drop(['@id', '@type', 'id', 'relation', 'address.district.@id', 'address.area.@id', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis=1)
    df1 = df1.rename(columns={'title':'Place of interest', 'address.locality':'City', 'address.street-address':'Place address', 'location.latitude':'lat_start', 'location.longitude':'long_start'})
    df1 ['Type of place'] = 'catolicas'
    return df1

In [6]:
df1 = pd.json_normalize(templos_catolicos['@graph'])
df1 = df1.drop(['@id', '@type', 'id', 'relation', 'address.district.@id', 'address.area.@id', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis=1)
df1 = df1.rename(columns={'title':'Place of interest', 'address.locality':'City', 'address.street-address':'Place address', 'location.latitude':'lat_start', 'location.longitude':'long_start'})
df1 ['Type of place'] = 'catolicas'


TypeError: 'function' object is not subscriptable

In [7]:
def templos_no_catolicos():
    templos_no_catolicos = requests.get('https://datos.madrid.es/egob/catalogo/209434-0-templos-otros.json')
    templos_no_catolicos = templos_no_catolicos.json()
    df2 = pd.json_normalize(templos_no_catolicos['@graph'])
    df2 = df2.drop(['@id', '@type', 'id', 'relation', 'address.district.@id', 'address.area.@id', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis=1)
    df2 = df2.rename(columns={'title':'Place of interest', 'address.locality':'City', 'address.street-address':'Place address', 'location.latitude':'lat_start', 'location.longitude':'long_start'})
    df2["Type of place"] = "no_catolicas"
    return df2

In [8]:
df2 = pd.json_normalize(templos_no_catolicos['@graph'])
df2 = df2.drop(['@id', '@type', 'id', 'relation', 'address.district.@id', 'address.area.@id', 'address.postal-code', 'organization.organization-desc', 'organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis=1)
df2 = df2.rename(columns={'title':'Place of interest', 'address.locality':'City', 'address.street-address':'Place address', 'location.latitude':'lat_start', 'location.longitude':'long_start'})
df2["Type of place"] = "no_catolicas"


TypeError: 'function' object is not subscriptable

In [9]:
def df3():
    df3 = pd.read_json("../../data/bicimad_stations.json")
    df3 ['LATITUD'] = [float(index.split(',')[1].replace("]", "")) for index in df3['geometry_coordinates']]
    df3 ['LONGITUD'] = [float(index.split(',')[0].replace("[", "")) for index in df3['geometry_coordinates']]
    df_bicimad = df3.drop(['id', 'light', 'number', 'activate', 'no_available', 'total_bases', 'dock_bikes', 'free_bases', 'reservations_count', 'geometry_type'], axis=1)
    df_bicimad = df_bicimad.rename(columns={'name': 'BiciMAD station', 'address': 'Station location', 'LONGITUD':'long_finish', 'LATITUD':'lat_finish'})
    df_bicimad ['final'] = df_bicimad.apply(lambda x: to_mercator(x['lat_finish'], x['long_finish']), axis =1)
    return df_bicimad

In [10]:
frames = [df1, df2]
df_origen = pd.concat(frames)
df_origen

NameError: name 'df1' is not defined

In [11]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [12]:
df_origen ['start'] = df_origen.apply(lambda x: to_mercator(x['lat_start'], x['long_start']), axis =1)

NameError: name 'df_origen' is not defined

In [13]:
df_origen

NameError: name 'df_origen' is not defined

In [14]:
df_merge = pd.merge(df_origen, df_bicimad, how = 'cross')
df_merge

NameError: name 'df_origen' is not defined

In [14]:
def distance_meters(start, final):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return start.distance(final)

In [15]:
df_merge ['Distance'] = df_merge.apply(lambda x: distance_meters(x['final'], x['start']), axis =1)
df_merge

,Place of interest,City,Place address,lat_start,long_start,Type of place,start,BiciMAD station,Station location,geometry_coordinates,long_finish,lat_finish,final,Distance
0,Basílica Jesús de Medinaceli,MADRID,PLAZA JESUS 2,40.414018,-3.695608,catolicas,POINT (4498867.944088976 -411678.7944271415),Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]",-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),780.283863
1,Basílica Jesús de Medinaceli,MADRID,PLAZA JESUS 2,40.414018,-3.695608,catolicas,POINT (4498867.944088976 -411678.7944271415),Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),762.661891
2,Basílica Jesús de Medinaceli,MADRID,PLAZA JESUS 2,40.414018,-3.695608,catolicas,POINT (4498867.944088976 -411678.7944271415),Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]",-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),1355.736981
3,Basílica Jesús de Medinaceli,MADRID,PLAZA JESUS 2,40.414018,-3.695608,catolicas,POINT (4498867.944088976 -411678.7944271415),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]",-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262),2207.690979
4,Basílica Jesús de Medinaceli,MADRID,PLAZA JESUS 2,40.414018,-3.695608,catolicas,POINT (4498867.944088976 -411678.7944271415),Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]",-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),1795.491308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140971,Unión De Mujeres Evangélicas UDME,MADRID,CALLE GABRIEL Y GALAN 26,40.451635,-3.669618,no_catolicas,POINT (4503055.45640462 -408779.5331972048),Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]",-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512),6459.906462
140972,Unión De Mujeres Evangélicas UDME,MADRID,CALLE GABRIEL Y GALAN 26,40.451635,-3.669618,no_catolicas,POINT (4503055.45640462 -408779.5331972048),Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]",-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427756),6458.374730
140973,Unión De Mujeres Evangélicas UDME,MADRID,CALLE GABRIEL Y GALAN 26,40.451635,-3.669618,no_catolicas,POINT (4503055.45640462 -408779.5331972048),Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]",-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274),6444.456655
140974,Unión De Mujeres Evangélicas UDME,MADRID,CALLE GABRIEL Y GALAN 26,40.451635,-3.669618,no_catolicas,POINT (4503055.45640462 -408779.5331972048),Facultad Derecho,Avenida Complutense nº 23,"[-3.72937, 40.45109]",-3.729370,40.451090,POINT (4502994.74083288 -415445.024322674),6665.767647


In [16]:
def bicimad_station():
    x = str(input('Introduce un sitio: '))
    y = df_merge[df_merge['Place of interest']== x]
    return y.sort_values(by='Distance', ascending=True).head(1).drop(['City', 'lat_start', 'long_start', 'start', 'geometry_coordinates', 'long_finish', 'lat_finish', 'final'], axis = 1)


In [17]:
bicimad_station()

Introduce un sitio: Basílica Jesús de Medinaceli


,Place of interest,Place address,Type of place,BiciMAD station,Station location,Distance
27,Basílica Jesús de Medinaceli,PLAZA JESUS 2,catolicas,Huertas,Calle Jesús nº 1,82.222184


0          762.661891
1          762.661891
2         1355.736981
3         2207.690979
4         1795.491308
             ...     
140971    6458.374730
140972    6458.374730
140973    6444.456655
140974    6665.767647
140975    2978.879015
Name: Distance, Length: 140976, dtype: float64

In [18]:
df_merge.sort_values(by = "Distance", ascending = True).groupby('Place of interest')['Type of place','Place address','BiciMAD station', 'Station location','Distance'].nth(0).drop(["Distance"], axis = "columns")


/Users/jorgediazserrano/miniconda3/envs/proyecto1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Type of place,Place address,BiciMAD station,Station location
Place of interest,,,,
Asambleas de Evangelización Mundial para Cristo (AEMC),no_catolicas,CALLE ANTONIA RODRIGUEZ SACRISTAN 8,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
Asociación Darse - Dar Ayuda para la Reinserción Social España,no_catolicas,CAMINO ALTO DE SAN ISIDRO 16 PLANTA 2º C C,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
Asociación Tiempo Para Cuidar,no_catolicas,CALLE GENERAL LACY 24 PLANTA BAJA A,Delicias,Calle Bustamante nº 1
Asociación Visión Internacional,no_catolicas,CALLE DISCOBOLO 53,Gutierre de Cetina,Calle Gutierre de Cetina nº 77
Basílica Jesús de Medinaceli,catolicas,PLAZA JESUS 2,Huertas,Calle Jesús nº 1
...,...,...,...,...
Testigos Cristianos de Jehová. Calle Puerto de Pozazal,no_catolicas,CALLE PUERTO DE POZAZAL 1 PLANTA BAJA,Pavones,Calle Hacienda de Pavones nº 342
Testigos Cristianos de Jehová. Calle San Moisés,no_catolicas,CALLE SAN MOISÉS 39 PLANTA BAJA B,Concordia,Calle Arroyo del Olivar nº 6
Testigos Cristianos de Jehová. Calle Tembleque,no_catolicas,CALLE TEMBLEQUE 64 PLANTA BAJA,Caramuel,Calle Caramuel nº 51


In [25]:
def bicimad_stations_group():
    x = str(input('Introduce un sitio: '))
    y = df_merge[df_merge['Place of interest']==x]
    return y.sort_values(by='Distance', ascending=True).drop(['City', 'lat_start', 'long_start', 'start', 'geometry_coordinates', 'long_finish', 'lat_finish', 'final'], axis = 1)


In [26]:
bicimad_stations_group()

Introduce un sitio: Unión De Mujeres Evangélicas UDME


,Place of interest,Place address,Type of place,BiciMAD station,Station location,Distance
140964,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Corazón de María,Calle Santa Hortensia nº 31,883.148484
140911,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Parque Berlín,Avenida de Concha Espina nº 34,941.249312
140878,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Colombia,Calle Colombia nº 7,976.280494
140910,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Prosperidad,Calle Suero de Quiñones nº 2,1037.172384
140854,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Doctor Arce 45,Avenida del Doctor Arce nº 45,1186.582080
...,...,...,...,...,...,...
140927,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Glorieta de Cádiz,Calle Antonio López nº 129,7765.904333
140930,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Puerta del Ángel,Plaza Puerta del Ángel nº 4,7787.892062
140937,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Ermita Santo,Paseo de la Ermita del Santo nº 48,7895.292009
140926,Unión De Mujeres Evangélicas UDME,CALLE GABRIEL Y GALAN 26,no_catolicas,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,7923.267172
